In [1]:
# Importing useful libraries

import pandas as pd 
import numpy as np
import os
import tensorflow as tf

In [2]:
img_size = 900

In [3]:
def build_decoder(with_labels=True, target_size=(img_size, img_size), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path) # Reads and outputs the entire contents of the input filename.

        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3) # Decode a PNG-encoded image to a uint8 or uint16 tensor
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3) # Decode a JPEG-encoded image to a uint8 tensor
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0 # Casts a tensor to the type float32 and divides by 255.
        img = tf.image.resize(img, target_size) # Resizing to target size
        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        img = tf.image.random_saturation(img, 0.8, 1.2)
        img = tf.image.random_brightness(img, 0.1)
        img = tf.image.random_contrast(img, 0.9, 1.2)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment

def build_dataset(paths, labels=None, bsize=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO) # overlaps data preprocessing and model execution while training
    return dset

In [4]:
model = tf.keras.models.load_model('../input/keras-ranzcr-clip/model.h5', custom_objects={'FixedDropout': tf.keras.layers.Dropout})

In [5]:
test_decoder = build_decoder(with_labels=False, target_size=(img_size, img_size))
load_dir = '/kaggle/input/ranzcr-clip-catheter-line-classification/'
df_sub = pd.read_csv(load_dir + 'sample_submission.csv')
test_paths = load_dir + "test/" + df_sub['StudyInstanceUID'] + '.jpg'
dtest = build_dataset(
    test_paths, bsize= 16, repeat=False, 
    shuffle=False, augment=False, cache=False, 
    decode_fn=test_decoder
)

In [6]:
y_preds = model.predict(dtest, verbose=1)

224/224 [==============================] - 269s 1s/step


In [7]:
df_sub.iloc[:, 1:] = y_preds
display(df_sub)

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present
0,1.2.826.0.1.3680043.8.498.46923145579096002617...,0.025917,0.313350,0.439144,0.019625,0.057158,3.263013e-02,0.834621,0.312184,0.192841,0.666376,0.978131
1,1.2.826.0.1.3680043.8.498.84006870182611080091...,0.000007,0.000029,0.000010,0.000067,0.000065,3.852217e-06,0.000002,0.012108,0.021624,0.984955,0.000001
2,1.2.826.0.1.3680043.8.498.12219033294413119947...,0.000055,0.000077,0.000009,0.000310,0.000190,1.668928e-06,0.000007,0.021027,0.310692,0.787104,0.000041
3,1.2.826.0.1.3680043.8.498.84994474380235968109...,0.011733,0.044406,0.041918,0.043582,0.054165,8.552958e-01,0.171819,0.262256,0.373406,0.501383,0.012053
4,1.2.826.0.1.3680043.8.498.35798987793805669662...,0.000377,0.003533,0.000721,0.001352,0.001579,9.184936e-06,0.001547,0.015810,0.050175,0.946002,0.000027
...,...,...,...,...,...,...,...,...,...,...,...,...
3577,1.2.826.0.1.3680043.8.498.81464483108873296584...,0.001620,0.001497,0.001021,0.034087,0.095443,8.264708e-01,0.251255,0.016031,0.091611,0.859840,0.000173
3578,1.2.826.0.1.3680043.8.498.33579133018211530710...,0.000227,0.000251,0.000029,0.000274,0.000381,7.826695e-07,0.001535,0.155265,0.648122,0.416685,0.001068
3579,1.2.826.0.1.3680043.8.498.61472811086105902907...,0.000039,0.000107,0.000026,0.000125,0.000126,1.187819e-06,0.000019,0.014523,0.188377,0.826021,0.000013
3580,1.2.826.0.1.3680043.8.498.19434375795525494655...,0.000030,0.000070,0.000050,0.000022,0.000062,7.903751e-07,0.000069,0.253844,0.718063,0.131176,0.000255


In [8]:
df_sub.to_csv('submission.csv',index=False)